# Exercises: compressed sensing 


Author: Stefano Pagani <stefano.pagani@polimi.it>.

Date: 2024

Course: Mathematical and numerical foundations of scientific machine learning.


In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cvx



Consider two different cases of reentrant activity (second one is commented).

In [ ]:
n=100
L=20

x=np.arange(-L,L,2*L/n)
y=x 

#print(x)
#print(y)

count_samples = 0

Xd=np.zeros((len(x)**2,10*n))
Td=np.zeros((len(x),len(x),10*n))
for ind_random in range(10):
    param = np.random.rand(1)
    print(param)
    for ind_sample in range(n):
        count = 0
        for ind_x in range(len(x)):
            for ind_y in range(len(y)):
                # case 1
                u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.5)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )                
                # case 2
                #u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.1+0.9*param)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )
                Xd[count,count_samples] = u
                count += 1

                Td[ind_x,ind_y,count_samples] = u
        
        count_samples += 1
            


Task 1: represent some snapshots of the reentrant activity at different timesteps. What is the effect of the parameter?

In [ ]:

plt.imshow( Td[:,:,230] )


Task 2: compute the SVD of the matrix and visualize the first basis functions.

In [ ]:

[U,S,V]=np.linalg.svd(Xd)


In [ ]:

#print(S)



In [ ]:

fig,axs = plt.subplots(2,1)
axs = axs.reshape(-1)

axs[0].plot(range(1000),100*(S)/np.sum(S),'o')

axs[1].semilogy(range(1000),100*(S)/np.sum(S),'o')



In [ ]:
plt.imshow( np.reshape(U[:,24],(n,n) ) )


Test set

In [ ]:

count_samples = 0

Xd_test=np.zeros((len(x)**2,5*n))
Td_test=np.zeros((len(x),len(x),5*n))
for ind_random in range(5):
    param = np.random.rand(1)
    print(param)
    for ind_sample in range(n):
        count = 0
        for ind_x in range(len(x)):
            for ind_y in range(len(y)):
                # case 1
                u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.5)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )                
                # case 2
                #u = np.tanh(np.sqrt(x[ind_x]**2+y[ind_y]**2) * np.cos(np.angle(x[ind_x]+y[ind_y]*(0+(1.0+param)*1j))-(0.1+0.9*param)*(np.sqrt(x[ind_x]**2+y[ind_y]**2))+ ind_sample/10) )
                
                Xd_test[count,count_samples] = u
                count += 1

                Td_test[ind_x,ind_y,count_samples] = u
        
        count_samples += 1

In [ ]:
perm = [ 50*100+7, 50*100+10, 50*100+17, 50*100+20, 
         57*100+7, 60*100+10, 67*100+17, 70*100+20,
         57*100-7, 60*100-10, 67*100-17, 70*100-20,
         43*100-7, 40*100-10, 33*100-17, 30*100-20,
         43*100+7, 40*100+10, 33*100+17, 30*100+20]

#p = 100

#perm = np.random.choice(n*n, size=p, replace=False)

#np.random.sample(np.arange(0,n,1), p)
#(np.floor(np.random.rand(p)*n)).astype(int)

p = len(perm)

C = np.zeros( (p,n*n) )
for i in range(p):
    C[i,perm[i]] = 1.0

u = Xd_test[:,233]

y = C @ u




Task 4: solve the reconstruction problem using compressed sensing. Consider the case where $\Psi$ is composed by all the solutions stored in Xd and the case where $\Psi$ is composed by the first 500 columns of U.

In [ ]:

## Solve compressed sensing problem
#Psi = U[:,:500] # Build Psi
Psi = Xd

Theta = C @ Psi                 # C * Psi

# print(Theta @ np.ones((100,)))

In [ ]:

# solving with CVX

# Create vector variables (CVXPY Variable)
s_c = cvx.Variable(shape=(np.shape(Psi)[1],))
#print(s_c)
# Create the constraints (Python list)
constraints = [Theta @ s_c == y]
# Form objective
obj = cvx.Minimize( cvx.norm(s_c,1) )
# Form and solve problem
prob = cvx.Problem(obj, constraints)
prob.solve()  # Returns the optimal value.
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", s_c.value)

urec = Psi @ s_c.value

In [ ]:
fig,axs = plt.subplots(1,3)


im0 = axs[0].imshow( np.reshape(u,(n,n) ) )
plt.colorbar(im0)

im1 = axs[1].imshow( np.reshape(urec,(n,n) ) )
plt.colorbar(im1)

im2 = axs[2].imshow( np.reshape(urec-u,(n,n) ) )
plt.colorbar(im2)
